In [194]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, confusion_matrix

# plot tree
import matplotlib.pyplot as plt
from sklearn import tree

In [248]:
# global variables
start_features = ["Age", "Department", "JobSatisfaction", "RelationshipSatisfaction", "MonthlyIncome", "Gender", "OverTime", "WorklifeBalance"]
target = "Attrition"


In [271]:
def main():
    # create dataframe with project's data set
    df = pd.read_csv('HR.csv')
    
    # clean the data
    clean_df = clean_data(df)
    
    # get features matrix X and target variable y
    X, y = feature_selection(clean_df)
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    # Create Decision Tree classifer object
    clf = DecisionTreeClassifier(max_depth=3, criterion='entropy') # use criterion=entropy as attribute or default gini

    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)

    # Predict the response for test dataset
    y_pred = clf.predict(X_test)
    """predict_proba: The first index refers to the probability 
    that the data belong to class 'No', and the second refers to 
    the probability that the data belong to class 'Yes'.""" 
    
    # Model Accuracy
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    # print(y_pred) 
    
    # Plot Tree
    plot_tree(clf, X.columns.tolist())
    

In [197]:
def extract_categoricals(df):
    cat_features = []
    
    # extract a list of all categorical features for feature engineering except target column
    for col_header in df.columns:
        if col_header != target and df.dtypes[col_header] == object:
            cat_features.append(col_header)         
    return cat_features
        

In [198]:
def clean_data(df):
    # drop columns that are not needed
    for col_header in df.columns:
        if col_header not in start_features and col_header != target:
            df.drop(col_header, axis=1, inplace=True)
            
    # drop rows that contain at least one empty cell
    df.dropna(axis='rows', thresh=1)
    
    # feature enigneering for categorical data
    clean_df = pd.get_dummies(data=df, columns=extract_categoricals(df))
    
    return clean_df


In [209]:
def feature_selection(clean_df):
    
    # get target variable and exclude it from features
    y = clean_df.pop(target)
      
    # get new feature list
    features = clean_df.columns.tolist()
    
    # split dataset in features and target variable
    X = clean_df[features] # Features Matrix
    
    return X, y


In [246]:
def plot_tree(clf, features):
    
    # Setting dpi = 300 to make image clearer than default
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(40,40), dpi=300)

    tree.plot_tree(clf,
           feature_names=features,
           class_names=["No","Yes"],
           filled=True,
           fontsize=12);

    fig.savefig('tree.png')


In [272]:
# test
if __name__ == '__main__':
    main()

Accuracy: 0.8390022675736961
[[345  19]
 [ 52  25]]
              precision    recall  f1-score   support

          No       0.87      0.95      0.91       364
         Yes       0.57      0.32      0.41        77

    accuracy                           0.84       441
   macro avg       0.72      0.64      0.66       441
weighted avg       0.82      0.84      0.82       441

